In this notebook, we'll perform conditional generation using the SpatialDIVA model, on the pancreatic cancer data from Zhou et al.

The results of this analysis will be a bit different from the paper result, as we'll only use 1000 samples (spots) for quick inference and demonstration purposes, and train the full model on a subset of the data (2 slides).

The high level and detailed processes for performing conditional generation is outlined in the manuscript. In general:

- We start by training the SpatialDIVA model on the data.
- We then use the trained model to generate latent samples for each of the factors we're considering.
- We zero out all but one of the factors, and generate samples considering only the variation of the remaining factor.

After this, we'll have generated transcriptomic counts specific to each factor, and we can perform additional downstream analyses, such as differential expression analysis, clustering, etc.

Let's start by loading the required libraries and the data.

In [4]:
%load_ext autoreload
%autoreload 2

import sys 
import os 
sys.path.append("..")

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc 
import anndata as ann 

from api import StDIVA

In [5]:
# Load the first two slides of the Zhou et al dataset
adata_path = "/projects/scDisent/adata_tumor"
adata_files = [f for f in os.listdir(adata_path) if f.endswith(".h5ad")]
adata_files = [os.path.join(adata_path, f) for f in adata_files]

adata_files_sub = adata_files[:2]
adatas = []
for adata_file in adata_files_sub:
    adata = sc.read_h5ad(adata_file)
    adatas.append(adata)

In [6]:
from sklearn.preprocessing import LabelEncoder

counts_dim = adata.shape[1] # Because we are using all the genes
uni_cols = [col for col in adatas[0].obs.columns if "UNI" in col]
hist_dim = len(uni_cols)
# When getting unique values, combine the celltypes from both slides
y1_dim = len(np.unique(np.concatenate([adatas[0].obs["ST_celltype"].values, adatas[1].obs["ST_celltype"].values])))
y2_dim = 100 # 50 PCs for the ST data and 50 PCs for the UNI data - neighbourhood context 

# Transform path labels due to string encoding and character issues
# When getting unique values, combine the pathologist annotations from both slides
path_labels = np.concatenate([adatas[0].obs["is_tumor"].values, adatas[1].obs["is_tumor"].values]) 
le = LabelEncoder()
path_labels = le.fit_transform(path_labels)

y3_dim = len(np.unique(path_labels))
d_dim = 2 # For two slides

In [7]:
stdiva = StDIVA(
    counts_dim = counts_dim,
    hist_dim = hist_dim,
    y1_dim = y1_dim,
    y2_dim = y2_dim,
    y3_dim = y3_dim,
    d_dim = d_dim,
    betas = [1, 1, 1, 1, 1, 1] # Default betas
)

In [13]:
stdiva.add_data(
    adata = adata_files[0:2],
    label_key_y1 = "ST_celltype",
    label_key_y3 = "is_tumor",
    hist_col_key = "UNI",
    hvg = False # We are not using HVGs - we are using all the genes
)

Processing data..


/h/hmaan/.cache/pypoetry/virtualenvs/spatialdiva-PyiMLd3V-py3.9/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/h/hmaan/.cache/pypoetry/virtualenvs/spatialdiva-PyiMLd3V-py3.9/lib/python3.9/site-packages/scipy/sparse/_data.py:149: RuntimeWarning: overflow encountered in expm1
  result = op(self._deduped_data())
/h/hmaan/.cache/pypoetry/virtualenvs/spatialdiva-PyiMLd3V-py3.9/lib/python3.9/site-packages/scipy/sparse/_data.py:149: RuntimeWarning: overflow encountered in expm1
  result = op(self._deduped_data())
/h/hmaan/.cache/pypoetry/virtualenvs/spatialdiva-PyiMLd3V-py3.9/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/h/hmaan/.cache/pypoetry/virtualenvs/spatialdiva-PyiMLd3V-py3.9/lib/python3.9/si

ValueError: cannot specify integer `bins` when input data contains infinity

In [12]:
stdiva.adata

AttributeError: 'StDIVA' object has no attribute 'adata'